In [1]:
include("../LiPoSID.jl")

#include("CtrlSID.jl")


using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra

using HDF5
using DynamicPolynomials

using Dates

using Statistics

using TSSOS

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} 0 & a \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$

$
J_2 = \begin{pmatrix} d & 0 \\ 0 & -d 
   \end{pmatrix} = 
   \begin{pmatrix} \sqrt{\gamma_2} & 0 \\ 0 & -\sqrt{\gamma_2} 
   \end{pmatrix}
$

$
J_3 = \begin{pmatrix} 0 & 0 \\ e & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 &  0 \\ \sqrt{\gamma_3} & 0 
   \end{pmatrix}
$

$
J_4 = \begin{pmatrix} 0 & p \\ p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3} 
   \begin{pmatrix} 0 & 1\\ 1 & 0 
   \end{pmatrix}
$

$
J_5 = \begin{pmatrix} 0 & -i p \\ i p & 0 
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 0 & -i \\ i & 0 
   \end{pmatrix}
$

$
J_6 = \begin{pmatrix} p & 0 \\ 0 & -p
   \end{pmatrix} = \sqrt{\gamma_4/3}
   \begin{pmatrix} 1 & 0 \\ 0 & -1 
   \end{pmatrix}
$

In [2]:
function frobenius_norm2(m)
    return tr(m * m')
end

frobenius_norm2 (generic function with 1 method)

In [3]:
function lindblad_rhs(ρ, H, J::Array)
    """
    Right hand side of the Lindblad master equation with multiple disipators
    """
   
    Σ = sum([ ( Jⱼ * ρ * Jⱼ' - (Jⱼ' * Jⱼ  * ρ + ρ * Jⱼ' * Jⱼ)/2 ) for Jⱼ in J ])
    
    return -im * (H * ρ - ρ * H) + Σ 
    
end

lindblad_rhs (generic function with 1 method)

In [4]:
function simpson_obj(ρ, t, H, J)
    
    obj = 0
    for i in 3:length(ρ)
        obj += frobenius_norm2(
            ρ[i] - ρ[i-2] - (t[i]-t[i-1])lindblad_rhs((ρ[i-2] + 4ρ[i-1] + ρ[i])/3, H, J)
        )
    end
    obj = sum(real(coef) * mon for (coef, mon) in zip(coefficients(obj), monomials(obj)))
    return obj
end

simpson_obj (generic function with 1 method)

In [5]:
function lindblad_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)

    ρᵍ, ρᵉ, ρˣ, ρʸ = ρᵍᵉˣʸ
    tᵍ, tᵉ, tˣ, tʸ = tᵍᵉˣʸ

    #[J₁ˢʸᵐᵇ, J₂ˢʸᵐᵇ, J₃ˢʸᵐᵇ, J₄ˢʸᵐᵇ, J₅ˢʸᵐᵇ, J₆ˢʸᵐᵇ] = Jˢʸᵐᵇ

    polyG = simpson_obj(ρᵍ, tᵍ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)
    polyE = simpson_obj(ρᵉ, tᵉ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)
    polyX = simpson_obj(ρˣ, tˣ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)
    polyY = simpson_obj(ρʸ, tʸ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)

    polyGEXY = polyG + polyE + polyX + polyY

    return polyGEXY
end

lindblad_GEXY_obj (generic function with 1 method)

In [6]:
# Define polynomial variables
@polyvar ϵ α β r ϕ e p

#Hˢʸᵐᵇ = [ ϵ   0
#          0   0. ]

Hˢʸᵐᵇ = [ ϵ            α + im* β
          α + im* β   -ϵ+0.0       ]/2

J₁ˢʸᵐᵇ = [ 0   r
           0   0. ]

J₂ˢʸᵐᵇ = ϕ * [ 1    0
               0   -1. ]

J₃ˢʸᵐᵇ = [ 0   0
           e   0. ]

J₄ˢʸᵐᵇ = p * [ 0    1
               1    0. ]

J₅ˢʸᵐᵇ = p * [ 0   -im
               im   0  ]

J₆ˢʸᵐᵇ = p * [ 1    0
               0   -1. ]


Jˢʸᵐᵇ = [ J₁ˢʸᵐᵇ, J₄ˢʸᵐᵇ, J₅ˢʸᵐᵇ, J₆ˢʸᵐᵇ] #J₃ˢʸᵐᵇ, J₂ˢʸᵐᵇ,

4-element Vector{Matrix}:
 Term{true, Float64}[0.0 r; 0.0 0.0]
 Term{true, Float64}[0.0 p; p 0.0]
 Term{true, Complex{Int64}}[(0 + 0im) (0 - 1im)p; (0 + 1im)p (0 + 0im)]
 Term{true, Float64}[p 0.0; 0.0 -p]

In [8]:
constraints = [ϵ, p] #r, e, ϕ, p]

2-element Vector{PolyVar{true}}:
 ϵ
 p

In [9]:
function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end


function read_GEXY_timeevolution(file_name, γ)

    tᵍ, ρᵍ = read_timeevolution(file_name, "B1", γ)
    tᵉ, ρᵉ = read_timeevolution(file_name, "B2", γ)
    tˣ, ρˣ = read_timeevolution(file_name, "B3", γ)
    tʸ, ρʸ = read_timeevolution(file_name, "B4", γ)

    ρᵍᵉˣʸ = ρᵍ, ρᵉ, ρˣ, ρʸ 
    tᵍᵉˣʸ = tᵍ, tᵉ, tˣ, tʸ

    return tᵍᵉˣʸ , ρᵍᵉˣʸ 

end

read_GEXY_timeevolution (generic function with 1 method)

In [10]:
function TrDist(ρ₁, ρ₂)
    A = ρ₁-ρ₂
    D = tr(sqrt(A'*A))/2
    if abs(imag(D))>1e-6
        throw(DomainError(D, "Trace distance is complex number"))
    else
        return(real(D))
    end
end 

TrDist (generic function with 1 method)

In [11]:
#tests_dir = "../TESTS/"
tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_states = vcat(train_states, test_states);

In [12]:
function filter_terms_by_relative_threshold(poly::Polynomial, relative_threshold::Float64)
    # Get all coefficients of the polynomial
    coeffs = coefficients(poly)
    
    # Find the largest coefficient by absolute value
    max_coeff = maximum(abs.(coeffs))
    
    # Calculate the effective threshold
    threshold = relative_threshold * max_coeff
    
    # Initialize an empty polynomial of the same type as the input
    new_poly = zero(poly)
    
    # Iterate over the terms and coefficients of the polynomial
    for (monomial, coeff) in zip(monomials(poly), coeffs)
        if abs(coeff) >= threshold
            new_poly += coeff * monomial
        end
    end
    
    return new_poly
end

# Example usage
#@polyvar x y
#p = 1e-13*x^2 + 2*x*y + 3*y^2 + 4e-14*y
#relative_threshold = 1e-12
#filtered_p = filter_terms_by_relative_threshold(p, relative_threshold)
#println(filtered_p)

filter_terms_by_relative_threshold (generic function with 1 method)

In [13]:

function filter_odd_terms_by_relative_threshold(poly::Polynomial, relative_threshold::Float64)
    # Get all coefficients and corresponding monomials of the polynomial
    coeffs = coefficients(poly)
    monoms = monomials(poly)
    
    # Find the largest coefficient by absolute value
    max_coeff = maximum(abs.(coeffs))
    
    # Calculate the effective threshold
    threshold = relative_threshold * max_coeff
    
    # Initialize an empty polynomial of the same type as the input
    new_poly = zero(poly)
    
    # Iterate over the terms and coefficients of the polynomial
    for (monomial, coeff) in zip(monoms, coeffs)
        # Check if the coefficient is above the threshold
        if abs(coeff) >= threshold
            # Add the term to the new polynomial
            new_poly += coeff * monomial
        else
            # Check if the monomial has only odd powers
            powers = exponents(monomial)
            if all(p -> p % 2 == 0, powers)
                # Add the term if it has even powers only
                new_poly += coeff * monomial
            end
        end
    end
    
    return new_poly
end

filter_odd_terms_by_relative_threshold (generic function with 1 method)

In [14]:
function coefficient_range(poly::Polynomial)
    # Extract coefficients of the polynomial
    coeffs = coefficients(poly)
    
    # Get the absolute values of the coefficients
    abs_coeffs = abs.(coeffs)
    
    # Find the minimum and maximum coefficients in terms of magnitude
    min_coeff = minimum(abs_coeffs)
    max_coeff = maximum(abs_coeffs)
    
    # Calculate the ratio of the smallest to the largest coefficient
    if max_coeff == 0
        return 0.0  # Avoid division by zero if all coefficients are zero
    else
        return min_coeff / max_coeff
    end
end

coefficient_range (generic function with 1 method)

In [15]:
evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

γᵢ = "0.25133"

tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)

((Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.086666666667, 0.093333333334, 0.1, 0.106666666667, 0.113333333334  …  23.806666666786, 23.813333333452, 23.820000000119, 23.826666666786, 23.833333333452, 23.840000000119, 23.846666666786, 23.853333333453, 23.860000000119, 23.866666666786], Any[0.053333333334, 0.06, 0.066666666667, 0.073333333334, 0.08, 0.08666666666

In [16]:

polyGEXYfull = lindblad_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)

0.18256809044653088r⁴ - 0.03750921946960905r²p² + 17.65754243030726p⁴ + 0.0012278271223203374αr² + 0.10341163322607885ϵ² - 4.104630886766604e-5ϵα + 1.0519096548020683α² + 1.0519096548020683β² - 0.09164160023232439r² + 0.0032558121009390465p² - 5.199679015672933ϵ + 0.00013669789552732257α + 65.37495321955478

In [17]:
coefficient_range(polyGEXYfull)

6.278598583438585e-7

In [18]:
function scaling_poly(p::Polynomial)
    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling))
end

scaling_poly (generic function with 1 method)

In [19]:
function min_cs_tssos_scaled(p, constrs)

    coeffs = coefficients(p)

    # find variable scaling
    scale = scaling_poly(p)

    # scale the polynomial
    p_scaled = subs(p, variables(p) => scale .* variables(p))
    constrs_scaled = subs.(constrs, variables(p) => scale .* variables(p))

    # minimize
    reg_coef = 0.0
    # minimizer_scaled_tssos = nothing
    pop =[p_scaled+reg_coef*sum(variables(p).^2), constrs_scaled...] ./ maximum(abs.(coeffs))

    d = maxdegree(p)
    
    # Initial optimization step
    opt, sol, data = cs_tssos_first(pop, variables(pop), d; solution=true, QUIET=true)
    ref_sol, flag = TSSOS.refine_sol(opt, sol, data; QUIET=true)
    prev_opt, prev_sol, prev_data = opt, sol, data 

    # Check if the solution needs further refinement
    if flag != 0
        while ~isnothing(sol) && flag != 0
            prev_opt, prev_sol, prev_data = opt, sol, data
            opt, sol, data = cs_tssos_higher!(data; solution=true, QUIET=true) 
        end
        ref_sol, flag = TSSOS.refine_sol(prev_opt, prev_sol, prev_data; QUIET=true)
    end

    minimizer_scaled_tssos = scale .* ref_sol

    solution = variables(p) => minimizer_scaled_tssos

    if flag == 0 
        status_name = "GLOBAL"
    else
        status_name = "LOCAL/FAIL"
    end

    return solution, status_name

end
  

min_cs_tssos_scaled (generic function with 1 method)

In [20]:
solution, status_name = min_cs_tssos_scaled(polyGEXYfull, constraints)
Hˢⁱᵈ = convert.(ComplexF64,subs(Hˢʸᵐᵇ, solution))
Hˢⁱᵈ[1:2,1:2]

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[2, 1]
[3, 1]
-----------------------------------------------------------------------------
optimum = 2.62232659867431e-5

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified with relative optimality gap 0.000020%!
Global optimality certified with relative optimality gap 0.000020%!


2×2 Matrix{ComplexF64}:
     12.5703+0.0im  0.000139521+0.0im
 0.000139521+0.0im     -12.5703+0.0im

In [21]:
function min_cs_tssos(p, constrs)

    coeffs = coefficients(p)
    reg_coef = 0

    pop =[p+reg_coef*sum(variables(p).^2), constrs...] ./ maximum(abs.(coeffs))

    d = maxdegree(p)
    
    # Initial optimization step
    opt, sol, data = cs_tssos_first(pop, variables(pop), d; solution=true, QUIET=true)
    ref_sol, flag = TSSOS.refine_sol(opt, sol, data; QUIET=true)
    prev_opt, prev_sol, prev_data = opt, sol, data 

    # Check if the solution needs further refinement
    if flag != 0
        while ~isnothing(sol) && flag != 0
            prev_opt, prev_sol, prev_data = opt, sol, data
            opt, sol, data = cs_tssos_higher!(data; solution=true, QUIET=true) 
        end
        ref_sol, flag = TSSOS.refine_sol(prev_opt, prev_sol, prev_data; QUIET=true)
    end

    solution = variables(p) => ref_sol

    if flag == 0 
        status_name = "GLOBAL"
    else
        status_name = "LOCAL/FAIL"
    end

    return solution, status_name

end

min_cs_tssos (generic function with 1 method)

In [22]:
sol, st = min_cs_tssos(polyGEXYfull, constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[2, 1]
[3, 1]
-----------------------------------------------------------------------------
termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = 5.281755737711362e-9
Global optimality certified with relative optimality gap 0.002602%!
Global optimality certified with relative optimality gap 0.002602%!


(PolyVar{true}[ϵ, α, β, r, p] => [25.140687132729255, 0.00027904154242244425, 0.0, 0.500996499173504, 0.013669226257621998], "GLOBAL")

In [23]:
Hˢⁱᵈ = convert.(ComplexF64,subs(Hˢʸᵐᵇ, sol))
Hˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
     12.5703+0.0im  0.000139521+0.0im
 0.000139521+0.0im     -12.5703+0.0im

In [26]:
sol, st = min_cs_tssos(filter_odd_terms_by_relative_threshold(polyGEXYfull, 1e-4), constraints)

*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[3, 1]
[1, 4]
-----------------------------------------------------------------------------
termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = 1.178506127727237e-9
Global optimality certified with relative optimality gap 0.002602%!
Global optimality certified with relative optimality gap 0.002602%!


(PolyVar{true}[ϵ, α, β, r, ϕ, p] => [25.14068705727399, 0.0, 0.0, 0.5009362324646646, 0.010409234788395123, 6.537495321954824e6, 0.013197464592987465], "GLOBAL")

In [27]:
st

"GLOBAL"

In [24]:
J₁ˢⁱᵈ = convert.(ComplexF64,subs(J₁ˢʸᵐᵇ, sol))
J₁ˢⁱᵈ[1:2,1:2]

2×2 Matrix{ComplexF64}:
 0.0+0.0im  0.500996+0.0im
 0.0+0.0im       0.0+0.0im

In [25]:
Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))

2×2 Matrix{ComplexF64}:
     12.5703+0.0im  0.000139521+0.0im
 0.000139521+0.0im     -12.5703+0.0im

In [27]:
println(" SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and LINDBLAD 5 Frobenius objective QO simulation")

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

relative_threshold = 1e-9

rltrs = string(convert(Int, floor(log10(relative_threshold))))

tests_data_file_name = "POP_NEW_TSSOS_LINDBLAD_4_CONSTR_treshold_1e"*rltrs*"_FROB_QO_"*date_and_time_string * ".h5"

#relative_threshold = 0
#tests_data_file_name = "POP_NEW_TSSOS_KOS_CONSTR_NOtreshold_FROB_QO_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Epsilons = []
CoefRanges = []
#Markovianity = []

for γᵢ in γ

    println("γ =  "*γᵢ)

    tᵍᵉˣʸ , ρᵍᵉˣʸ  = read_GEXY_timeevolution(evol_data_file_name, γᵢ)
    polyGEXYfull = lindblad_GEXY_obj(ρᵍᵉˣʸ, tᵍᵉˣʸ, Hˢʸᵐᵇ, Jˢʸᵐᵇ)

    push!(CoefRanges, coefficient_range(polyGEXYfull))

    #polyGEXY = polyGEXYfull
    polyGEXY = filter_odd_terms_by_relative_threshold(polyGEXYfull, relative_threshold)

    @show minimum(abs.(coefficients(polyGEXY)))
    @show maximum(abs.(coefficients(polyGEXY)))

    #sol, opt_val, status = CtrlSID.cs_tssos(polyGEXY, constraints)
    sol, status = min_cs_tssos(polyGEXY, constraints)
    #min_cs_tssos_scaled

    print(" status:", status)

    Hˢⁱᵈ = convert.(ComplexF64, subs(Hˢʸᵐᵇ, sol))

    Jˢⁱᵈ = [convert.(ComplexF64,subs(Jᵢˢʸᵐᵇ, sol)) for Jᵢˢʸᵐᵇ in Jˢʸᵐᵇ]

    epsilon = subs(ϵ, sol)

    push!(Epsilons, epsilon)
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   
        γ_group["epsilon"] = convert(Float64, epsilon)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)
        γ_group["J1"] = convert.(ComplexF64, Jˢⁱᵈ[1])
        γ_group["J2"] = convert.(ComplexF64, Jˢⁱᵈ[2])
        γ_group["J3"] = convert.(ComplexF64, Jˢⁱᵈ[3])
        γ_group["J4"] = convert.(ComplexF64, Jˢⁱᵈ[4])
        #γ_group["J5"] = convert.(ComplexF64, Jˢⁱᵈ[5])
        #γ_group["J"] = convert.(ComplexF64, Jˢⁱᵈ)
        γ_group["status"] = status

    end

    println()

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        #bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        ρₒ = DenseOperator(basis, ρₛ[1])
        dt = tᵗˢᵗ[2] - tᵗˢᵗ[1]
        tᵉⁿᵈ = tᵗˢᵗ[end]

        #print("effective_Lindblad_ops for Kossakowski")

        #effective_Lindblad = get_lindblad_operators(convert.(ComplexF64, Cˢⁱᵈ), fᴷᴼᴺᴮ)
        effective_Lindblad_ops = [DenseOperator(basis,j) for j in Jˢⁱᵈ]

        #print("Simulating Kossakowski")

        tout, ρ_t_lindblad = timeevolution.master(convert.(Float64, tᵗˢᵗ), ρₒ, DenseOperator(basis, Hˢⁱᵈ), effective_Lindblad_ops)
        ρˢⁱᵈ  = [ρₜ.data for ρₜ in ρ_t_lindblad]

        #bˢⁱᵈ = LiPoSID.bloch(ρˢⁱᵈ)

        F = LiPoSID.fidelity_series(basis, ρₛ, ρˢⁱᵈ)

        #Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, F)
            #init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            #init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
            init_state_group["time"] = tᵗˢᵗ
        end
        
        FminState = minimum(F)
        FmedianState = mean(F)
        FmeanState = mean(F)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)
    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

h5open(tests_dir*tests_data_file_name,"cw") do fid
    fid["F_min"] = convert.(Float64,FminGammas)
    fid["F_median"] = convert.(Float64,FmedianGammas)
    fid["F_mean"] = convert.(Float64,FmeanGammas)
    fid["Energy"] = convert.(Float64,Epsilons)
    fid["CoefsRanges"] = convert.(Float64,CoefRanges)

end


println(tests_data_file_name)

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and LINDBLAD 5 Frobenius objective QO simulation
γ =  0.079477
minimum(abs.(coefficients(polyGEXY))) = 0.00010716999563515663
maximum(abs.(coefficients(polyGEXY))) = 501.8627949039175
*********************************** TSSOS ***********************************
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[2, 1]
[3, 1]
-----------------------------------------------------------------------------
termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = 8.889384496794077e-9
Global optimality certified with relative optimality gap 0.000260%!
Global optimality certified with relative optimality gap 0.000260%!
 status:GLOBAL
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9996334039455367
γ =  0.25133
minimum(abs.(coefficients(polyGEXY))) = 4.104630886766604e-5
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478
****************

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective QO simulation
γ =  0.079477
minimum(abs.(coefficients(polyGEXY))) = 0.00010716999563516075
maximum(abs.(coefficients(polyGEXY))) = 501.8627949039175
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9976226185525447
γ =  0.25133
minimum(abs.(coefficients(polyGEXY))) = 3.45175532950157e-5
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.996633883661821
γ =  0.79477
minimum(abs.(coefficients(polyGEXY))) = 2.1434036477811963e-6
maximum(abs.(coefficients(polyGEXY))) = 7.630493479021682
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.997693174205029
γ =  2.5133
minimum(abs.(coefficients(polyGEXY))) = 1.780563581016409e-7
maximum(abs.(coefficients(polyGEXY))) = 1.121995426822635
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9990491303024693
γ =  7.9477
minim

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective QO simulation
γ =  0.079477
minimum(abs.(coefficients(polyGEXY))) = 7.16715101396591e-37
maximum(abs.(coefficients(polyGEXY))) = 501.8627949039175
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9976226185525441
γ =  0.25133
minimum(abs.(coefficients(polyGEXY))) = 4.658513032068184e-37
maximum(abs.(coefficients(polyGEXY))) = 65.37495321955478
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9966338836618186
γ =  0.79477
minimum(abs.(coefficients(polyGEXY))) = 5.527952391752069e-38
maximum(abs.(coefficients(polyGEXY))) = 7.630493479021682
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9976931742050117
γ =  2.5133
minimum(abs.(coefficients(polyGEXY))) = 9.073762994668326e-39
maximum(abs.(coefficients(polyGEXY))) = 1.121995426822635
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9990491303025519
γ =  7.9477
mi

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective QO simulation
γ =  0.079477
 status:Local
D1 effective_Lindblad_opsSimulating KossakowskiD2 effective_Lindblad_opsSimulating KossakowskiD3 effective_Lindblad_opsSimulating KossakowskiD4 effective_Lindblad_opsSimulating KossakowskiD5 effective_Lindblad_opsSimulating KossakowskiD6 effective_Lindblad_opsSimulating KossakowskiD7 effective_Lindblad_opsSimulating KossakowskiD8 effective_Lindblad_opsSimulating KossakowskiD9 effective_Lindblad_opsSimulating KossakowskiD10 effective_Lindblad_opsSimulating KossakowskiMedian fidelity for 0.079477: 0.9976226185525441
γ =  0.25133
 status:Local
D1 effective_Lindblad_opsSimulating KossakowskiD2 effective_Lindblad_opsSimulating KossakowskiD3 effective_Lindblad_opsSimulating KossakowskiD4 effective_Lindblad_opsSimulating KossakowskiD5 effective_Lindblad_opsSimulating KossakowskiD6 effective_Lindblad_opsSimulating KossakowskiD7 effective_Lindblad_opsSimulating KossakowskiD8 effective_

 SYSTEM IDENTIFICATION w CONSTRAINED TSSOS and KOSSAKOWSKI objective
γ =  0.079477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9977046690410768
γ =  0.25133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9966807880531685
γ =  0.79477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9977004864050036
γ =  2.5133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.999049835240695
γ =  7.9477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9996748517732994
γ =  25.133
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9968645845347491
γ =  79.477
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.9989415677938539
γ =  251.33
 status:Local
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.999393310891379
POP_NEW_TSSOS_KOS_2024-Jun-15_at_04-22.h5


In [18]:
println(tests_data_file_name)

POP_NEW_TSSOS_KOS_2024-Jun-15_at_04-22.h5
